In [1]:
import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._ 
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split
import org.apache.spark.graphx.EdgeDirection


Intitializing Scala interpreter ...

Spark Web UI available at http://mbpdejebaptiste:4040
SparkContext available as 'sc' (version = 3.0.1, master = local[*], app id = local-1618523076483)
SparkSession available as 'spark'


import org.apache.spark._
import org.apache.spark.graphx._
import org.apache.spark.rdd.RDD
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window
import org.apache.spark.sql.Row
import org.apache.spark.sql.functions.split
import org.apache.spark.graphx.EdgeDirection


## Load des données

Données Karate

In [2]:
val karate_text = sc.textFile("data/soc-karate/soc-karate.mtx")


karate_text: org.apache.spark.rdd.RDD[String] = data/soc-karate/soc-karate.mtx MapPartitionsRDD[1] at textFile at <console>:39


Données Facebook

In [3]:
//choix du fichier
var file_number = "0"
var facebook_text = sc.textFile("data/facebook_data/facebook/" + file_number + ".edges")


file_number: String = 0
facebook_text: org.apache.spark.rdd.RDD[String] = data/facebook_data/facebook/0.edges MapPartitionsRDD[3] at textFile at <console>:41


## Preprocess

In [4]:
var df_karate_text = karate_text.toDF.withColumn("id",monotonicallyIncreasingId)


df_karate_text = df_karate_text.withColumn("rank", row_number().over(Window.orderBy("id")))
df_karate_text = df_karate_text.filter(df_karate_text("rank")>24)
df_karate_text = df_karate_text.drop("id","rank")

val df_karate_splitted = df_karate_text.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

val rows: RDD[Row] = df_karate_splitted.rdd

var karate_edges = rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong,1)}


df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_text: org.apache.spark.sql.DataFrame = [value: string]
df_karate_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]
rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[13] at rdd at <console>:52
karate_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[14] at map at <console>:54


In [5]:
var facebook_text = sc.textFile("data/facebook_data/facebook/" + file_number + ".edges")
var df_facebook_text = facebook_text.toDF()

var df_facebook_text_splitted = df_facebook_text.withColumn("_tmp", split($"value", "\\ ")).select(
  $"_tmp".getItem(0).as("srcId"),
  $"_tmp".getItem(1).as("dstID"),
)

val facebook_rows: RDD[Row] = df_facebook_text_splitted.rdd

var facebook_edges = facebook_rows.map{ case Row(src:String, dist : String) => Edge(src.toLong, dist.toLong,1)}


facebook_text: org.apache.spark.rdd.RDD[String] = data/facebook_data/facebook/0.edges MapPartitionsRDD[16] at textFile at <console>:42
df_facebook_text: org.apache.spark.sql.DataFrame = [value: string]
df_facebook_text_splitted: org.apache.spark.sql.DataFrame = [srcId: string, dstID: string]
facebook_rows: org.apache.spark.rdd.RDD[org.apache.spark.sql.Row] = MapPartitionsRDD[21] at rdd at <console>:50
facebook_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[22] at map at <console>:52


## Creation du graphe

In [6]:
var chosen_edges = karate_edges

chosen_edges: org.apache.spark.rdd.RDD[org.apache.spark.graphx.Edge[Int]] = MapPartitionsRDD[14] at map at <console>:54


In [7]:
var default_user=(0, 0)
var graph = Graph.fromEdges(chosen_edges, default_user)

default_user: (Int, Int) = (0,0)
graph: org.apache.spark.graphx.Graph[(Int, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@512ed8c3


In [8]:
graph.edges.collect()

res0: Array[org.apache.spark.graphx.Edge[Int]] = Array(Edge(2,1,1), Edge(3,1,1), Edge(3,2,1), Edge(4,1,1), Edge(4,2,1), Edge(4,3,1), Edge(5,1,1), Edge(6,1,1), Edge(7,1,1), Edge(7,5,1), Edge(7,6,1), Edge(8,1,1), Edge(8,2,1), Edge(8,3,1), Edge(8,4,1), Edge(9,1,1), Edge(9,3,1), Edge(10,3,1), Edge(11,1,1), Edge(11,5,1), Edge(11,6,1), Edge(12,1,1), Edge(13,1,1), Edge(13,4,1), Edge(14,1,1), Edge(14,2,1), Edge(14,3,1), Edge(14,4,1), Edge(17,6,1), Edge(17,7,1), Edge(18,1,1), Edge(18,2,1), Edge(20,1,1), Edge(20,2,1), Edge(22,1,1), Edge(22,2,1), Edge(26,24,1), Edge(26,25,1), Edge(28,3,1), Edge(28,24,1), Edge(28,25,1), Edge(29,3,1), Edge(30,24,1), Edge(30,27,1), Edge(31,2,1), Edge(31,9,1), Edge(32,1,1), Edge(32,25,1), Edge(32,26,1), Edge(32,29,1), Edge(33,3,1), Edge(33,9,1), Edge(33,15,1), Edge(33...


In [9]:
graph.vertices.collect()

res1: Array[(org.apache.spark.graphx.VertexId, (Int, Int))] = Array((20,(0,0)), (13,(0,0)), (19,(0,0)), (34,(0,0)), (15,(0,0)), (4,(0,0)), (21,(0,0)), (16,(0,0)), (22,(0,0)), (25,(0,0)), (28,(0,0)), (29,(0,0)), (11,(0,0)), (14,(0,0)), (32,(0,0)), (30,(0,0)), (24,(0,0)), (27,(0,0)), (33,(0,0)), (23,(0,0)), (1,(0,0)), (6,(0,0)), (17,(0,0)), (3,(0,0)), (7,(0,0)), (9,(0,0)), (8,(0,0)), (12,(0,0)), (18,(0,0)), (31,(0,0)), (26,(0,0)), (10,(0,0)), (5,(0,0)), (2,(0,0)))


### Calcul des degrés des noeuds

In [10]:
val newgraph = graph.outerJoinVertices(graph.degrees)((index,_,deg) => (index.toInt,deg.get))

newgraph: org.apache.spark.graphx.Graph[(Int, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@157f2644


In [11]:
newgraph.triplets.collect()

res2: Array[org.apache.spark.graphx.EdgeTriplet[(Int, Int),Int]] = Array(((2,(2,9)),(1,(1,16)),1), ((3,(3,10)),(1,(1,16)),1), ((3,(3,10)),(2,(2,9)),1), ((4,(4,6)),(1,(1,16)),1), ((4,(4,6)),(2,(2,9)),1), ((4,(4,6)),(3,(3,10)),1), ((5,(5,3)),(1,(1,16)),1), ((6,(6,4)),(1,(1,16)),1), ((7,(7,4)),(1,(1,16)),1), ((7,(7,4)),(5,(5,3)),1), ((7,(7,4)),(6,(6,4)),1), ((8,(8,4)),(1,(1,16)),1), ((8,(8,4)),(2,(2,9)),1), ((8,(8,4)),(3,(3,10)),1), ((8,(8,4)),(4,(4,6)),1), ((9,(9,5)),(1,(1,16)),1), ((9,(9,5)),(3,(3,10)),1), ((10,(10,2)),(3,(3,10)),1), ((11,(11,3)),(1,(1,16)),1), ((11,(11,3)),(5,(5,3)),1), ((11,(11,3)),(6,(6,4)),1), ((12,(12,1)),(1,(1,16)),1), ((13,(13,2)),(1,(1,16)),1), ((13,(13,2)),(4,(4,6)),1), ((14,(14,5)),(1,(1,16)),1), ((14,(14,5)),(2,(2,9)),1), ((14,(14,5)),(3,(3,10)),1), ((14,(14,5...


In [12]:
newgraph.vertices.collect()

res3: Array[(org.apache.spark.graphx.VertexId, (Int, Int))] = Array((20,(20,3)), (13,(13,2)), (19,(19,2)), (34,(34,17)), (15,(15,2)), (4,(4,6)), (21,(21,2)), (16,(16,2)), (22,(22,2)), (25,(25,3)), (28,(28,4)), (29,(29,3)), (11,(11,3)), (14,(14,5)), (32,(32,6)), (30,(30,4)), (24,(24,5)), (27,(27,2)), (33,(33,12)), (23,(23,2)), (1,(1,16)), (6,(6,4)), (17,(17,2)), (3,(3,10)), (7,(7,4)), (9,(9,5)), (8,(8,4)), (12,(12,1)), (18,(18,2)), (31,(31,4)), (26,(26,3)), (10,(10,2)), (5,(5,3)), (2,(2,9)))


ETAPE 1: Variables utiles pour calculer le gain en modularité lors de la suppression d'un noeud d'une communauté

In [13]:
def sigmaIn1(i:Int, comId: Int, graph : Graph[(Int, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1==comId) &&
                                                    !(triplet.srcId==i || triplet.dstId==i)))
    return graphVar.count.toDouble

}


def sigmaTot1(i:Int, comId: Int, graph : Graph[(Int, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => 
    (triplet.srcAttr._1==comId && triplet.srcId !=i && triplet.dstAttr._1 !=comId) || 
    (triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId && triplet.dstId !=i) ||
    (triplet.srcId==i && triplet.dstAttr._1==comId) || 
    (triplet.srcAttr._1==comId && triplet.dstId==i ) 
    ))
    return graphVar.count.toDouble
}



sigmaIn1: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double
sigmaTot1: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double


ETAPE 2: Variables utiles pour calculer le gain en modularité lors de l'ajout d'un noeud à une communauté

In [14]:
def sigmaIn2(comId: Int, graph : Graph[(Int, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcAttr._1==comId && triplet.dstAttr._1==comId))
    return graphVar.count.toDouble
}


def sigmaTot2(comId: Int, graph : Graph[(Int, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcAttr._1==comId && triplet.dstAttr._1!=comId)||(triplet.srcAttr._1!=comId && triplet.dstAttr._1==comId)))
    return graphVar.count.toDouble
}

sigmaIn2: (comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double
sigmaTot2: (comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double


Variables utiles au 2 étapes

In [15]:
def ki(i: Int, graph : Graph[(Int, Int),Int]): Double = {
    var graphVar = graph.triplets.filter((triplet => triplet.srcId==i || triplet.dstId==i))
    return graphVar.count.toDouble
}

def kiin(i:Int, comId : Int, graph : Graph[(Int, Int),Int]) : Double = {
    var graphVar = graph.triplets.filter((triplet => (triplet.srcId==i && triplet.dstAttr._1==comId) ||(triplet.srcAttr._1==comId && triplet.dstId==i)))
    return graphVar.count.toDouble
}

ki: (i: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double
kiin: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double


Calcul du différentiel de modularité

In [16]:
def dq(sigmain:Double, sigmatot:Double, kivar:Double, kiinvar:Double, m:Double):Double = {
    return (((sigmain+kiinvar)/(2*m) - scala.math.pow((sigmatot+kivar)/(2*m),2)) - (sigmain/(2*m) - scala.math.pow(sigmatot/(2*m),2) - scala.math.pow(kivar/(2*m),2)))
}

dq: (sigmain: Double, sigmatot: Double, kivar: Double, kiinvar: Double, m: Double)Double


In [17]:
def maxCom(setDq : Set[(Int, Double)]) : (Int, Double) = {
    var max = -1000.0
    var ind = -1
    setDq.foreach(x => {
        if (x._2>max)
        {
            ind=x._1
            max=x._2
        }
    })
    return (ind,max)
}

maxCom: (setDq: Set[(Int, Double)])(Int, Double)


In [18]:
def moveItoC(i:Int, comId : Int, graph:Graph[(Int, Int),Int]) : Graph[(Int, Int),Int] = {
    val newVertices = graph.mapVertices { case (id, attr) => if (id==i) (comId, attr._2) else attr }
    return newVertices
}

moveItoC: (i: Int, comId: Int, graph: org.apache.spark.graphx.Graph[(Int, Int),Int])org.apache.spark.graphx.Graph[(Int, Int),Int]


In [19]:
def q(graph:Graph[(Int,Int),Int]) : Double = {
    var m = graph.edges.count()
    var vertices = graph.vertices.collect
    var q=0.0
    vertices.foreach{ case t1 => {
        vertices.foreach{ case t2 => {

            var isEdge = graph.edges.filter(e => (e.srcId == t1._1 &&e.dstId==t2._1) || (e.dstId==t2._1 && e.srcId==t1._1)).count.toDouble
            if (t1._2._1 == t2._2._1){
                q+= isEdge - t1._2._2.toDouble*t2._2._2.toDouble/(2*m)
            }
            

            }
        }   
    
        }
    }
    return q/(2*m)
}

q: (graph: org.apache.spark.graphx.Graph[(Int, Int),Int])Double


In [23]:
def getVertexI(a : (VertexId, (Int, Int)), b : (VertexId, (Int, Int))): (Int, Int) = {
            if (a._1 == 1) a else b
            }

newgraph.vertices.reduce(getVertexI)

<console>: 40: error: type mismatch;

In [33]:
def communities(graph:Graph[(Int,Int),Int]) : (Graph[(Int,Int),Int],Boolean) = {
    var currentGraph = graph
    val nbVertices = graph.vertices.count
    val m = graph.edges.count

    var i = 0
    var keepGoing = true
    //var j=0
    
    //println(j)
    keepGoing=false
    println("nombre actuel de communautées: "+((currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct).size)
    for (i<- 1 to nbVertices.toInt){
        println("Noeud considéré : "+i+ '\n')
        /*get the community and ki */
        /*def getVertexI(a: (Int, Int), b: (Int, Int)): (Int, Int) = {
            if (a._1 == i) a else b
            }*/
        val vertice = currentGraph.vertices.filter{case (id, t) => id == i}.map{case (id, t) => t}.collect
        val comId = vertice(0)._1
        println("Communauté de "+i+" : "+comId)
        val kivar = vertice(0)._2

        /* calculate dq1 */
        val kiin1 = kiin(i, comId, currentGraph)
        val sigmain1 = sigmaIn1(i, comId, currentGraph)
        val sigmatot1 = sigmaTot1(i, comId, currentGraph)
        val dq1 = dq(sigmain1, sigmatot1, kivar, kiin1, m)

        /*look all combis : we need to know how many communities are left and which one*/
        //val communities_available = (currentGraph.vertices.collect { case t => t._2._1 }).collect.distinct
        val direction = EdgeDirection.Either 
        val communities_available = currentGraph.collectNeighbors(direction).lookup(i)(0).map{case (id, t) => t._1}
        var setDq =Set(): Set[(Int,Double)]
        communities_available.foreach( a => {
            if (a!=comId){
                val sigmain2 = sigmaIn2(a, currentGraph)
                val sigmatot2 = sigmaTot2(a, currentGraph)
                val kiin2 = kiin(i,a, currentGraph)
                val dq2 = dq(sigmain2, sigmatot2, kivar, kiin2, m)
                setDq+=((a.toInt,dq2-dq1))
            }
        })

        /* get best combi and move if larger than 0 */
        val bestCombi = maxCom(setDq)
        if (bestCombi._2.toDouble > 0.0001 ) {
            
            println("On met "+ i +" dans "+bestCombi._1+ '\n')
            var updatedGraph = moveItoC(i, bestCombi._1, currentGraph)
            currentGraph=updatedGraph

            /*Un changement est fait*/
            keepGoing=true

        }
        println("Q = "+q(currentGraph))
    }
    println("Q = "+q(currentGraph))
    return (currentGraph, keepGoing)
}

communities: (graph: org.apache.spark.graphx.Graph[(Int, Int),Int])(org.apache.spark.graphx.Graph[(Int, Int),Int], Boolean)


In [ ]:
var keep_going = true
var updated_graph = newgraph
var iteration = 0

while (keep_going){
    iteration += 1 
    println("ITERATION: "+iteration)
    var community_tuple = communities(updated_graph)
    updated_graph = community_tuple._1
    keep_going = community_tuple._2
    if (iteration>0){
        keep_going = false}
}

ITERATION: 1
nombre actuel de communautées: 34
Noeud considéré : 1

Communauté de 1 : 1
On met 1 dans 12

Q = -0.04470742932281392
Noeud considéré : 2

Communauté de 2 : 2
On met 2 dans 22

Q = -0.03977646285338591
Noeud considéré : 3

Communauté de 3 : 3
On met 3 dans 10

Q = -0.035009861932938834
Noeud considéré : 4

Communauté de 4 : 4
On met 4 dans 13

Q = -0.02958579881656804
Noeud considéré : 5

Communauté de 5 : 5
On met 5 dans 11

Q = -0.02391518737672583
Noeud considéré : 6

Communauté de 6 : 6
On met 6 dans 17

Q = -0.018162393162393157
Noeud considéré : 7

Communauté de 7 : 7
On met 7 dans 17

Q = -0.007314266929651547
Noeud considéré : 8

Communauté de 8 : 8
On met 8 dans 13

Q = -0.0035338593030900724
Noeud considéré : 9

Communauté de 9 : 9
On met 9 dans 31

Q = 0.0012327416173570011
Noeud considéré : 10

Communauté de 10 : 10
Q = 0.0012327416173570011
Noeud considéré : 11

Communauté de 11 : 11
Q = 0.0012327416173570011
Noeud considéré : 12

Communauté de 12 : 12
Q = 0.0

### TEST

On veut savoir s'il est intéressant de placer le noeud 1 dans la communauté 12

In [197]:
var sigmain2 = sigmaIn2(12,newgraph)
var sigmatot2 = sigmaTot2(12,newgraph)
var kivar = ki(1, newgraph)
var kiinvar = kiin(1,12,newgraph)
var m=newgraph.edges.count

sigmain2: Double = 0.0
sigmatot2: Double = 1.0
kivar: Double = 16.0
kiinvar: Double = 1.0
m: Long = 78


In [ ]:
var dq1 = dq(sigmain2, sigmatot2, kivar, kiinvar, m)

In [198]:
var dq2 = dq(sigmain2, sigmatot2, kivar, kiinvar, m)

dq2: Double = 0.00509533201840894


In [199]:
var a = q(newgraph)

a: Double = -0.04980276134122287


In [200]:
var graphCible = moveItoC(1,12,newgraph)

graphCible: org.apache.spark.graphx.Graph[(Int, Int),Int] = org.apache.spark.graphx.impl.GraphImpl@648efdb2


In [201]:
var b = q(graphCible)

b: Double = -0.04470742932281392


In [202]:
a+dq2

res51: Double = -0.04470742932281393


In [203]:
var sigmain2 = sigmaIn2(13,graphCible)
var sigmatot2 = sigmaTot2(13,graphCible)
var kivar = ki(1, graphCible)
var kiinvar2 = kiin(1,13,graphCible)
var m=graphCible.edges.count.toDouble
var dq2 = dq(sigmain2, sigmatot2, kivar, kiinvar2, m)

var sigmain1 = sigmaIn1(1,12,graphCible)
var sigmatot1 = sigmaTot1(1,12,graphCible)
var kiinvar1 = kiin(1,12,graphCible)

var dq1= dq(sigmain1, sigmatot1, kivar, kiinvar1, m)

var dqtot = dq2 - dq1

println(dqtot)



-0.0013149243918474714


sigmain2: Double = 0.0
sigmatot2: Double = 2.0
kivar: Double = 16.0
kiinvar2: Double = 1.0
m: Double = 78.0
dq2: Double = 0.003780407626561469
sigmain1: Double = 0.0
sigmatot1: Double = 1.0
kiinvar1: Double = 1.0
dq1: Double = 0.00509533201840894
dqtot: Double = -0.0013149243918474714


In [23]:
var communities_available = newgraph.collectNeighbors(direction).lookup(1)(0).map{case (id, t) => t._1}

communities_available: Array[Int] = Array(2, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 18, 20, 22, 32)


In [22]:
        val direction = EdgeDirection.Either 


direction: org.apache.spark.graphx.EdgeDirection = EdgeDirection.Either


In [25]:
newgraph.edges.toDF.show(78)

+-----+-----+----+
|srcId|dstId|attr|
+-----+-----+----+
|    2|    1|   1|
|    3|    1|   1|
|    3|    2|   1|
|    4|    1|   1|
|    4|    2|   1|
|    4|    3|   1|
|    5|    1|   1|
|    6|    1|   1|
|    7|    1|   1|
|    7|    5|   1|
|    7|    6|   1|
|    8|    1|   1|
|    8|    2|   1|
|    8|    3|   1|
|    8|    4|   1|
|    9|    1|   1|
|    9|    3|   1|
|   10|    3|   1|
|   11|    1|   1|
|   11|    5|   1|
|   11|    6|   1|
|   12|    1|   1|
|   13|    1|   1|
|   13|    4|   1|
|   14|    1|   1|
|   14|    2|   1|
|   14|    3|   1|
|   14|    4|   1|
|   17|    6|   1|
|   17|    7|   1|
|   18|    1|   1|
|   18|    2|   1|
|   20|    1|   1|
|   20|    2|   1|
|   22|    1|   1|
|   22|    2|   1|
|   26|   24|   1|
|   26|   25|   1|
|   28|    3|   1|
|   28|   24|   1|
|   28|   25|   1|
|   29|    3|   1|
|   30|   24|   1|
|   30|   27|   1|
|   31|    2|   1|
|   31|    9|   1|
|   32|    1|   1|
|   32|   25|   1|
|   32|   26|   1|
|   32|   29

In [26]:
var ceci = newgraph.collectNeighbors(direction).lookup(1)(0)

ceci: Array[(org.apache.spark.graphx.VertexId, (Int, Int))] = Array((2,(2,9)), (3,(3,10)), (4,(4,6)), (5,(5,3)), (6,(6,4)), (7,(7,4)), (8,(8,4)), (9,(9,5)), (11,(11,3)), (12,(12,1)), (13,(13,2)), (14,(14,5)), (18,(18,2)), (20,(20,3)), (22,(22,2)), (32,(32,6)))
